In [17]:
import pymongo
import json
from tqdm import tqdm

# Connect to your MongoDB instance
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["vaers"]
col = db["reports cleaned"]
target_db = client["vaers"]
target_col = target_db["combinations cleaned"]

In [18]:
with open('/home/sebastian/Documents/Masterarbeit/Combinations/All_Combinations_cleaned_Filtered.json') as f:
    data = json.load(f)

In [19]:
for item in tqdm(data, desc="Processing items", unit="item"):
    N = col.estimated_document_count()
    
    D_query = {
        "vax_data": {
            "$elemMatch": {
                "VAX_MANU": item["manufacturer"],
                "VAX_TYPE": item["vaccine"]
            }
        }
    }
    D = col.count_documents(D_query)
    
    E_query = {
        "symptoms": {
            "$elemMatch": {
                "$in": [item["Event"]]
            }
        }
    }
    E = col.count_documents(E_query)
    
    DE_query = {'$and': [D_query, E_query]}
    DE = col.count_documents(DE_query)
    
    De = D - DE
    dE = E - DE
    de = N - (DE + De + dE)
    
    entry = {
        "vaccine": item["vaccine"],
        "manufacturer": item["manufacturer"],
        "symptom": item["Event"],  # Assuming the symptom key is "Event" in your data
        "DE": DE,
        "dE": dE,
        "De": De,
        "de": de
    }
    
    # Insert the entry into the MongoDB collection
    target_col.insert_one(entry)


Processing items: 100%|██████████| 143761/143761 [22:37:21<00:00,  1.77item/s]   
